In [39]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from dotenv import load_dotenv

load_dotenv('.env')

True

In [40]:
from crewai import LLM
groq_llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.3,
    max_tokens=4096,
    api_key=os.environ.get("GROQ_API_KEY"),
  
)


In [41]:
rag_tool = PDFSearchTool(pdf='/home/himanshu/Downloads/drone_finalest_final.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)
 

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2024-11-12 00:23:27,856 - 132296989983808 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--000bcd28c2824ab28b4c86b1d29e170d375f9d3529b843627021fdbeba5354d6
2024-11-12 00:23:27,857 - 132296989983808 - sqlite.py-sqlite:297 - WARNING: Insert of existing embedding ID: default-app-id--000bcd28c2824ab28b4c86b1d29e170d375f9d3529b843627021fdbeba5354d6
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00, 83.02it/s]


In [42]:
if "TAVILY_API_KEY" not in os.environ:
  os.environ["TAVILY_API_KEY"] = "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"
web_search_tool = TavilySearchResults(k=3)
     

web_search_tool.run("What does Sporo Health do?")

[{'url': 'https://www.linkedin.com/pulse/how-sporo-health-can-help-hospitals-reduce-costs-improve-ckbee',
  'content': "Sporo Health's technology reduces the time spent on documentation while ensuring that providers have access to accurate and up-to-date patient information, which can prevent errors and improve the"},
 {'url': 'https://www.linkedin.com/company/sporo-health',
  'content': 'At Sporo Health, ethical AI is at the heart of what we do, ensuring that every step forward in healthcare is a step towards a more equitable and safe future for all.'},
 {'url': 'https://sporohealth.com/sporo-health-develops-ai-for-healthcare-to-combat-physician-burnout/',
  'content': "AI solutions can be integrated within all EHRs using API documentation. At Sporo Health, we have a dedicated team of engineers and AI experts who will integrate SPORO's automation tools into any electronic medical record (EMR) software and tailor them to the physician's workflow needs. We are EHR agnostic and can integ

In [43]:
@tool
def router_tool(question):
  """Router Function"""
  if 'Sporo Health' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [44]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=groq_llm,
)

In [45]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=groq_llm,
)

In [46]:


Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=groq_llm,
)
     

hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
)
     

answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
)


In [47]:


router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)
     

retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)
     

grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)
     

hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)
     


In [48]:


rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)
     

inputs ={"question":"Does Sporo Streamline patient chart reviews?"}
     

result = rag_crew.kickoff(inputs=inputs)
     


2024-11-12 00:23:59,511 - 132296989983808 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Router
## Task: Analyse the keywords in the question Does Sporo Streamline patient chart reviews?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.
 

I encountered an error while trying to use the tool. This was the error: router_tool() missing 1 required positional argument: 'question'.
 Tool router_tool accepts these inputs: router_tool() - Router Function 



# Agent: Router
## Thought: Thought: The question seems to be related to a specific product or service, and the keywords suggest a question about its functionality.
## Using tool: router_tool
## Tool Input: 
"{\"question\": \"Does Sporo Streamline patient chart reviews?\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: router_tool() missing 1 